# Basic (delimited) continuations

Let's start by giving the `Functor`, `Applicative` and `Monad` instances for the continuation type constructor. Using an infix operator instead of the standard `runCont` helps reduce boilerplate.

In [33]:
{-# LANGUAGE InstanceSigs #-}

newtype Cont r a = Cont { (>>-) :: (a -> r) -> r }

instance Functor (Cont r) where
    fmap :: (a -> b) -> Cont r a -> Cont r b
    fmap f m = Cont $ \k -> m >>- (k . f)
    
instance Applicative (Cont r) where
    pure :: a -> Cont r a
    pure a = Cont $ \k -> k a
    (<*>) :: Cont r (a -> b) -> Cont r a -> Cont r b
    m <*> n = Cont $ \k -> m >>- \f -> n >>- \x -> k $ f x
    
instance Monad (Cont r) where
    return :: a -> Cont r a
    return = pure
    (>>=) :: Cont r a -> (a -> Cont r b) -> Cont r b
    m >>= k = Cont $ \l -> m >>- \x -> k x >>- \y -> l y

## The continuation applicative implies the continuation monad 

It turns out that the `Applicative` instance for `Cont` already implies the `Monad` instance. In the `Cont` monad, the `join m` is equivalent to `m . pure` (minus operations for dealing with the `runCont` boilerplate. This is illustrated below.

- TODO prove that this is equivalent to join as standardly defined.

In [34]:
join :: Cont r (Cont r a) -> Cont r a
join m = Cont $ (((>>-) <$> m) >>-) . (>>-) . pure

# Indexed continuations

Indexed continuations drop the presupposition that what Shan calls the "answer types" are identical.

- `ixPure` is defined in exactly the same way as the `pure` of `Cont`; answer types must be identical.
- The definition of `ixAp` is really *exactly* the same as the definition of the `<*>` of the `Cont` applicative, only the type is more general. What is necessary is that the intermediate result type of the first argument match the final result type of the second argument; these types cancel out.

In [49]:
ixMap :: (a -> b) -> IxCont r i a -> IxCont r i b
ixMap f m = IxCont $ \k -> m >>- \x -> k $ f x


ixPure :: a -> IxCont r r a
ixPure a = IxCont $ \k -> k a

ixAp :: IxCont r r' (a -> b) -> IxCont r' r'' a -> IxCont r r'' b
m `ixAp` n = IxCont $ \k -> m >>- \f -> n >>- \x -> k $ f x

## The *in

- Question: can we define an `ixJoin` in terms of the applicative operations?
- Answer: yes, the same trick will work, just so long as the intermediate type of the outer continuation layer matches the final result type of the inner continuation layer.

In [50]:
ixJoin :: IxCont r'' r (IxCont r r' a) -> IxCont r'' r' a
ixJoin m = IxCont $ (((>>-) `ixMap` m) >>-) . (>>-) . ixPure